In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
import requests
from bs4 import BeautifulSoup
drive.mount('/content/drive')

Mounted at /content/drive


### 업종(소분류, 중분류) 불러오기 - NAVER 증권 기준

In [ ]:
stock_name_df = []
single_stock_df = []
stocks = []

In [ ]:
#네이버 증권의 업종 분류(소분류) 가져오는 함수 만들기
def stock_append(i):
  url = "https://finance.naver.com/sise/sise_group_detail.nhn?type=upjong&no="+str(i)

  res = requests.get(url)
  soup = BeautifulSoup(res.text, 'lxml')
  stock_name = soup.find("table", attrs={"class": "type_1"}).find_all('td')[1].text.strip() #업종명
  stock_list = soup.find("table", attrs={"class": "type_5"}).find("tbody").find_all('tr') #그 업종에 속한 종목 리스트
  for stock in stock_list:
    if len(stock) > 1 :
      single_stock = stock.get_text().strip().split()[0]
      stock_name_df.append(stock_name)
      single_stock_df.append(single_stock)
      stocks.append([stock_name, single_stock])

#업종들 페이지들을 크롤링하여 정보 가져오기(소분류)
for i in range(261,339):
  stock_append(i)
stock_append(25)

In [ ]:
len(stock_name_df)

3303

In [ ]:
stock_name_pd = pd.DataFrame(data = stock_name_df, columns = ['업종'])
single_stock_pd = pd.DataFrame(data = single_stock_df, columns = ['회사명'])
df_stock = pd.DataFrame(data = stocks, columns = ['업종','회사명'])

In [ ]:
df_stock = df_stock.iloc[list(single_stock_pd.drop_duplicates().index),:].reset_index()

In [ ]:
df_stock.to_csv('/content/drive/MyDrive/upjong.csv')

In [ ]:
df_stock

,index,업종,회사명
0,0,제약,안트로젠
1,1,제약,팜젠사이언스
2,2,제약,국전약품
3,3,제약,휴메딕스
4,4,제약,셀트리온헬스케어
...,...,...,...
2593,3298,기타,유니포인트
2594,3299,기타,이엠티
2595,3300,기타,명진홀딩스
2596,3301,기타,지앤이헬스케어


In [ ]:
#업종 중분류 딕셔너리로 만들기(네이버 증권 기준)
upjong_dict = {'에너지':['에너지장비및서비스','석유와가스'],'소재':['화학','포장재','비철금속','철강','종이와목재'],
      '자본재':['우주항공과국방','건축제품','건축자재','건설','가구','전기장비','복합기업','기계','조선', '무역회사와판매업체'],
      '상업서비스와공급품': ['상업서비스와공급품'], '운송':['항공화물운송과물류', '항공사', '해운사', '도로와철도운송', '운송인프라'],
      '자동차와부품':['자동차부품', '자동차'],'내구소비재와의류':['가정용기기와용품', '레저용장비와제품', '섬유,의류,신발,호화품', '화장품', '문구류'],
      '호텔,레스토랑,레저등':['호텔,레스토랑,레저'],'미디어':['광고', '방송과엔터테인먼트', '출판', '양방향미디어와서비스'],
      '소매':['판매업체', '인터넷과카탈로그소매', '백화점과일반상점', '전문소매'],'교육서비스':['교육서비스'],
      '식품과기본식료품소매': ['식품과기본식료품소매'],'식품,음료,담배':['식품', '담배', '음료'],'가정용품과개인용품':['가정용품'],
      '건강관리장비와서비스':['건강관리장비와용품', '건강관리업체및서비스', '건강관리기술'],'제약과생물공학':['생물공학', '제약', '생명과학도구및서비스'],
      '은행':['은행'],'증권':['증권'],'다각화된금융':['창업투자', '카드', '기타금융'],'보험':['손해보험', '생명보험'],'부동산':['부동산'],
      '소프트웨어와서비스':['소프트웨어', 'IT서비스', '게임엔터테인먼트'],'기술하드웨어와장비':['통신장비', '핸드셋', '컴퓨터와주변기기', '전자장비와기기', '사무용전자제품'],
      '반도체와반도체장비':['반도체와반도체장비'],'전자와전기제품':['전자제품', '전기제품'],'디스플레이':['디스플레이패널','디스플레이장비및부품'],
      '통신서비스':['무선통신서비스','다각화된통신서비스'],'유틸리티':['가스유틸리티', '전기유틸리티', '복합유틸리티', '독립전력생산및에너지거래'],'기타':['기타']}

In [ ]:
#위의 딕셔너리 dataframe으로 만들기
upjong_list = []
for key, values in upjong_dict.items():
  for value in values:
    upjong_list.append([key,value])
middle_df = pd.DataFrame(upjong_list, columns = ['중분류','소분류'])
middle_df

In [ ]:
#위에서 구한 업종(소분류)와 종목 dataframe을 중분류와 합치기
upjong = pd.merge(df_stock, middle_df, how = 'left', left_on = '업종', right_on = '소분류')[['회사명','중분류','소분류']]

In [ ]:
upjong.head()

In [ ]:
upjong.shape

In [ ]:
#업종 정보 저장해두기
upjong.to_csv('/content/drive/MyDrive/upjong.csv')